In [9]:
import asyncio
from typing import Sequence, Tuple, Iterable, List
from tqdm.auto import trange

import torch
import hivemind
import petals

from petals.server.handler import TransformerConnectionHandler, split_for_streaming
from petals.client import RemoteSequenceManager, ClientConfig
from petals.client.remote_forward_backward import DEFAULT_MAX_MSG_SIZE, iter_as_aiter, aiter_with_timeout, deserialize_tensor_stream
from petals.data_structures import ModuleUID, PeerID, CHAIN_DELIMITER, UID_DELIMITER
from petals.utils.packaging import pack_args_kwargs, unpack_args_kwargs

from hivemind.compression import serialize_torch_tensor
from hivemind.utils import MSGPackSerializer, nested_flatten
from hivemind.proto import runtime_pb2

_END_OF_STREAM_KEY = "_EOS"


async def _run_forward_part(
    uid: str, serialized_tensors: Iterable[runtime_pb2.Tensor], stub, config: ClientConfig, metadata: dict
) -> List[torch.Tensor]:
    """Send (serialized) inputs to run forward pass as per rpc_forward_backward; return model outputs"""
    assert _END_OF_STREAM_KEY not in metadata
    parts = [tensor_part for tensor in serialized_tensors
             for tensor_part in split_for_streaming(tensor, DEFAULT_MAX_MSG_SIZE)]
    if len(parts) > 1:
        serialized_metadata = MSGPackSerializer.dumps(metadata)
    serialized_metadata_last_piece = MSGPackSerializer.dumps(dict(metadata, **{_END_OF_STREAM_KEY: True}))
    
    requests = [
        runtime_pb2.ExpertRequest(
            uid=uid, tensors=[tensor_part], 
            metadata=serialized_metadata if i != len(parts) - 1 else serialized_metadata_last_piece)
        for i, tensor_part in enumerate(parts)
    ]
        
    print("CALLING stub.rpc_forward_stream on serialized inputs", requests)
    outputs = await asyncio.wait_for(stub.rpc_forward_backward_stream(iter_as_aiter(requests)), config.connect_timeout)
    outputs = aiter_with_timeout(outputs, config.request_timeout)
    return await deserialize_tensor_stream(msg.tensors async for msg in outputs)



async def run_remote_forward_backward(
    sequence_manager: RemoteSequenceManager,
    peer_id: PeerID,
    span_uids: Sequence[ModuleUID],
    *args: torch.Tensor,
    **kwargs: torch.Tensor,
) -> Tuple[torch.Tensor, ...]:
    """
    Serializes input tensors and calls "rpc_forward_backward" on a remote server.
    Mostly adapted from https://github.com/learning-at-home/hivemind/blob/7a7c93aefffc9494c39e7b170c07cb06d8c09c4c/hivemind/moe/client/expert.py#L198
    but without RemoteExpertWorker.run_coroutine() call that leads to deadlock here.
    """
    merged_uid = CHAIN_DELIMITER.join(span_uids)
    stub = TransformerConnectionHandler.get_stub(sequence_manager.state.p2p, peer_id)
    flat_tensors, args_structure = pack_args_kwargs(*args, **kwargs)
    metadata = sequence_manager.get_request_metadata("rpc_forward", args_structure, uids=span_uids, *args, peer_id=peer_id, **kwargs) #TODO fix metadata api
    #codecs = sequence_manager.get_compression_codecs(peer_id, "rpc_forward", span_uids, *args, **kwargs)
    codecs = [runtime_pb2.CompressionType.NONE for _ in args]  #TODO replace with proper compression
    flat_tensors = tuple(tensor.cpu().detach().requires_grad_(tensor.requires_grad) for tensor in flat_tensors)
    args_structure = metadata.setdefault("args_structure", args_structure)
    if codecs is None:
        codecs = [runtime_pb2.CompressionType.NONE] * len(flat_tensors)
    else:
        codecs = list(nested_flatten(codecs))
        assert len(codecs) == len(flat_tensors), f"got {len(flat_tensors)} tensors but {len(codecs)} compression codecs"

    # Asynchronous serialization
    loop = asyncio.get_running_loop()
    serialized_tensors = await asyncio.gather(
        *(
            loop.run_in_executor(None, serialize_torch_tensor, tensor, compression)
            for tensor, compression in zip(flat_tensors, codecs)
        )
    )

    # call RPC on remote server
    size = sum(t.element_size() * t.nelement() for t in flat_tensors)
    # Hotfix: we use "// 2" since hivemind==1.1.5 serializes bfloat16 tensors in float32, so they take 2x more space - TODO remove in the next PR
    output_tensors = await _run_forward_part(
        merged_uid, serialized_tensors, stub, sequence_manager.config, metadata=metadata
    )
    # backward compatibility: ensure requires_grad; remove after https://github.com/learning-at-home/hivemind/pull/591
    requires_grad = any(tensor.requires_grad for tensor in flat_tensors)
    output_tensors = [tensor.requires_grad_(requires_grad) for tensor in output_tensors]
    return output_tensors


In [10]:
INITIAL_PEERS = ['/ip4/127.0.0.1/tcp/1337/p2p/QmRTdR9XmTHNXKiwtqRJ4i7tNofnmFrxkufBefguZUyXej']
peer_id_string = INITIAL_PEERS[0].split("/")[-1]
model_name = "Maykeye/TinyLLama-v0"

model_config = petals.DistributedLlamaConfig.from_pretrained(model_name)
block_uids = [
    f"{model_config.dht_prefix}{UID_DELIMITER}{i}"
    for i in range(model_config.num_hidden_layers)
]

block_in_use = block_uids[0:2]


try:
    dht = hivemind.DHT(start=True, client_mode=True, initial_peers=INITIAL_PEERS)
    sequence_manager = petals.RemoteSequenceManager(model_config, block_uids,  dht=dht)
    sequence_manager.rpc_info
    p2p = await dht.replicate_p2p()
    
    dummy_inputs = [torch.rand(1, 128, model_config.hidden_size, dtype=model_config.torch_dtype),
                    torch.empty(0, dtype=model_config.torch_dtype)]
    peer_id = hivemind.PeerID.from_base58(peer_id_string)
    for i in trange(10):
        (outputs,) = await run_remote_forward_backward(sequence_manager, peer_id, block_in_use, *dummy_inputs)
        print('outputs:', repr(outputs)[:50], '...')
    print("It works!")

finally:
    print("shutting down")
    await p2p.shutdown()
    dht.shutdown()  # it is okay to remove this clause, but you will be summoning a horde of daemons as you debug

Mar 09 22:09:16.235 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Mar 09 22:09:16.236 [INFO] Using DHT prefix: TinyLLama-v0-hf
100%|██████████| 10/10 [00:00<00:00, 63.18it/s]

CALLING stub.rpc_forward_stream on serialized inputs [uid: "TinyLLama-v0-hf.0 TinyLLama-v0-hf.1"
tensors {
  buffer: "$?o?\030>\306>m?\003?,?(?\374>\330=}?\004>\017?[?D?\034?$>\266>\240<L?-?\310=\\?\300<\003?w?\260=\226>p>m?I?\\?\024?c?E? >k?R?v?\322>\272>\242>\206>\200;Q?0=z?<?&?\332>\300=\212>@?\314>F?\033?\200=\276>\026?\230=\r?\202>\250=\324>\202>=?$?\034>V?\023?\222>\036?g?\\>\020?o?\340=~?\366>d?\340>l>x?h>\216>\000>,?f?\270>\320>\230>\340<2?[?I?\360>w?[?\340<@>7?0=\276>\270>\206>\013?`=\300>|>\202>h>0>\000?\350=g?\274>\374>\270=\204>=?\350>\244>\036?&?(?O?~?\316>\202>#?.?\010>\005?v?n?\314>R?4>\200<\214>h> ?6?\034>P?D>~?\022?\364>j?\006?g?\240<\025?1?h?\024>\020?\334>`>\004>T>??\026?Z?B?z?\236>i?-?\020=\260>D><?\n?=?\272>\372>\360>\222>\236>\202>\364>2?\234>X?\266>\216>\021?\316>\300>L>\'?.?D>\230>9?\352>\017?-?4?\024>|>B?\260=\020=\350=x?M?:?\242>\037?$?H>a?\322>b?1?O?\302>\256> >\324>L?\033?\000\000\260=E?,?o?h?N?z?]?*?\014>E?\\?C?4?f?\033?\220=\346>R?\332>c?E?)?\260=!?\026?\0